<a href="https://colab.research.google.com/github/welingtongomes/Projeto-Automacao_Vencimento_Contratos/blob/main/Controle_CRES_VCTO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [180]:
import pandas as pd
import datetime
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication



In [181]:
df = pd.read_csv('/content/_SELECT_sfh_servidor_id_sfh_data_fim_funcao_s_funcionario_ptr_id_202306301557.csv')


In [182]:
df['data_fim_funcao'] = pd.to_datetime(df['data_fim_funcao'])


In [183]:
#dia_atual = datetime.date.today()


In [184]:
# Calcular a data de vencimento de contratos nos próximos 15 dias
#data_vencimento = dia_atual + datetime.timedelta(days=15)


In [185]:
# Filtrar os contratos com vencimento nos próximos 15 dias
#dados_filtrados = df[df['data_fim_funcao'] <= data_vencimento]

In [186]:
# Calcular a data atual
data_atual = datetime.date.today()

# Calcular a data de vencimento de contratos nos próximos 15 dias
data_vencimento = data_atual + datetime.timedelta(days=15)

# Filtrar os contratos com vencimento nos próximos 15 dias
dados_filtrados = df[pd.to_datetime(df['data_fim_funcao']).dt.date <= data_vencimento]
dados_filtrados = dados_filtrados[pd.to_datetime(dados_filtrados['data_fim_funcao']).dt.date >= data_atual]



In [187]:
# Ordena os contratos de forma ascendente
dados_filtrados = dados_filtrados.sort_values('data_fim_funcao')

In [188]:
# Ajusta a coluna com vencimento de contrato para o padrão de data utilizado no Brasil
dados_filtrados['data_fim_funcao'] = dados_filtrados['data_fim_funcao'].dt.strftime('%d-%m-%Y')

In [189]:
# Renomeia a coluna data_fim_funcao para Vencimento de Contrato
dados_filtrados = dados_filtrados.rename(columns={'data_fim_funcao': 'Vencimento de Contrato'})


In [190]:
if not dados_filtrados.empty:
    dados_filtrados = dados_filtrados.drop(['servidor_id', 'funcionario_ptr_id'], axis=1)


# Salva o DataFrame como um arquivo Excel
    contratos_vcto = '/content/arquivo.xlsx'
    dados_filtrados.to_excel(contratos_vcto, index=False)

# Configura as informações do email
    mensagem = f"""\
    Prezados(as),

    Segue, em anexo, relatório com os dados referente ao vencimento de contratos de Docentes e Agentes CRES nos próximos 15 dias.

    Atenciosamente,
    Welington Henrique de Almeida Gomes
                """
    remetente = 'welington.gomes@uenp.edu.br'
    senha_remetente = 'Digite Sua Senha'
    destinatario = 'welington.gomes@outlook.com'
    assunto = 'Relatório de Vencimento de Contratos CRES - Próximos 15 dias'

# Cria um objeto MIMEMultipart para o email
    msg = MIMEMultipart()
    msg['From'] = remetente
    msg['To'] = destinatario
    msg['Subject'] = assunto

# Faz a leitura do arquivo Excel em formato binário
    with open(contratos_vcto, 'rb') as excel_file:
      excel_data = excel_file.read()

# Cria um objeto MIME application/vnd.openxmlformats-officedocument.spreadsheetml.sheet
    excel_mime = MIMEApplication(excel_data, _subtype='vnd.openxmlformats-officedocument.spreadsheetml.sheet')

# Define o nome do arquivo Excel anexado
    excel_mime.add_header('Content-Disposition', 'attachment', filename='relatorio.xlsx')

# Anexa o arquivo Excel ao email
    msg.attach(excel_mime)
# Adicionaa a mensagem ao corpo do email
    msg.attach(MIMEText(mensagem, 'plain'))

# Enviaa o email usando o servidor SMTP
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login(remetente, senha_remetente)
    server.send_message(msg)
    server.quit()
